In [1]:
from bs4 import BeautifulSoup
import requests
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

In [24]:
import pprint

In [2]:
my_url = 'http://www.nytimes.com'

In [3]:
r = requests.get(my_url)

In [4]:
r.text

u'<!DOCTYPE html>\n<!--[if (gt IE 9)|!(IE)]> <!--> <html lang="en" class="no-js edition-domestic app-homepage"  itemscope xmlns:og="http://opengraphprotocol.org/schema/"> <!--<![endif]-->\n<!--[if IE 9]> <html lang="en" class="no-js ie9 lt-ie10 edition-domestic app-homepage" xmlns:og="http://opengraphprotocol.org/schema/"> <![endif]-->\n<!--[if IE 8]> <html lang="en" class="no-js ie8 lt-ie10 lt-ie9 edition-domestic app-homepage" xmlns:og="http://opengraphprotocol.org/schema/"> <![endif]-->\n<!--[if (lt IE 8)]> <html lang="en" class="no-js lt-ie10 lt-ie9 lt-ie8 edition-domestic app-homepage" xmlns:og="http://opengraphprotocol.org/schema/"> <![endif]-->\n<head>\n    <title>The New York Times - Breaking News, World News & Multimedia</title>\n        <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\n<link rel="shortcut icon" href="https://static.nytimes.com/favicon.ico" />\n<link rel="apple-touch-icon-precomposed" sizes="144\xd7144" href="https://static.nytimes.com/images/i

In [5]:
soup = BeautifulSoup(r.text, "html.parser")

In [6]:
print soup.prettify()

<!DOCTYPE html>
<!--[if (gt IE 9)|!(IE)]> <!-->
<html class="no-js edition-domestic app-homepage" itemscope="" lang="en" xmlns:og="http://opengraphprotocol.org/schema/">
 <!--<![endif]-->
 <!--[if IE 9]> <html lang="en" class="no-js ie9 lt-ie10 edition-domestic app-homepage" xmlns:og="http://opengraphprotocol.org/schema/"> <![endif]-->
 <!--[if IE 8]> <html lang="en" class="no-js ie8 lt-ie10 lt-ie9 edition-domestic app-homepage" xmlns:og="http://opengraphprotocol.org/schema/"> <![endif]-->
 <!--[if (lt IE 8)]> <html lang="en" class="no-js lt-ie10 lt-ie9 lt-ie8 edition-domestic app-homepage" xmlns:og="http://opengraphprotocol.org/schema/"> <![endif]-->
 <head>
  <title>
   The New York Times - Breaking News, World News &amp; Multimedia
  </title>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <link href="https://static.nytimes.com/favicon.ico" rel="shortcut icon"/>
  <link href="https://static.nytimes.com/images/icons/ios-ipad-144x144.png" rel="apple-touch-icon-prec

In [7]:
paragraphs = soup.find_all("p", class_="summary")

In [8]:
stories_to_read = []
for item in paragraphs:
    stories_to_read.append(item.get_text())

In [9]:
print(stories_to_read)

[u'President Trump and members of his cabinet aggressively defended his policy of separating children from their parents at the border in response to a bipartisan outcry.\n\u201cThey could be murderers and thieves and so much else,\u201d Mr. Trump said of people crossing the border. He also repeated the false claim that Democrats were to blame for the policy.', u'\n        Attorney General Jeff Sessions and Stephen Miller, the president\u2019s top policy adviser, once shouted from the sidelines. Now they\u2019re the driving force behind the policy separating immigrant children from parents.    ', u'The audio recording, released by the news site ProPublica, captured the sounds of 10 Central American children who were separated from their parents by immigration authorities.', u'President Trump escalated a trade dispute with Beijing, saying the administration would identify another $200 billion worth of goods that could face tariffs.', u'Kim Jong-un, North Korea\u2019s leader, arrived on 

In [10]:
sentiments_list = []
for story in stories_to_read:
    analysis_subject = TextBlob(story, analyzer = NaiveBayesAnalyzer())
    sentiments_list.append(analysis_subject.sentiment)

In [11]:
print sentiments_list

[Sentiment(classification='pos', p_pos=0.9959044349723524, p_neg=0.0040955650276407836), Sentiment(classification='pos', p_pos=0.9998072869128124, p_neg=0.00019271308719245726), Sentiment(classification='pos', p_pos=0.9826894613637395, p_neg=0.017310538636261537), Sentiment(classification='pos', p_pos=0.9457180046261427, p_neg=0.05428199537385236), Sentiment(classification='pos', p_pos=0.8761631225697204, p_neg=0.12383687743028272), Sentiment(classification='pos', p_pos=0.8387626224406045, p_neg=0.16123737755939355), Sentiment(classification='pos', p_pos=0.9966127484154698, p_neg=0.003387251584531808), Sentiment(classification='pos', p_pos=0.5786896918046832, p_neg=0.4213103081953132), Sentiment(classification='pos', p_pos=0.8479876966084553, p_neg=0.1520123033915443), Sentiment(classification='neg', p_pos=0.42070198943909115, p_neg=0.5792980105609092), Sentiment(classification='neg', p_pos=0.43974317793516976, p_neg=0.5602568220648259), Sentiment(classification='pos', p_pos=0.95404911

In [12]:
for item in analysis_subject.noun_phrases:
    print item

new generation
real estate investors
hgtv
renovation isn ’ t


In [13]:
for story in stories_to_read:
    analysis_subject = TextBlob(story, analyzer = NaiveBayesAnalyzer())
    for item in analysis_subject.noun_phrases:
        print item

trump
bipartisan outcry
mr. trump
jeff sessions
stephen miller
president ’ s
top policy adviser
’ re
immigrant children
news site
propublica
central american children
immigration authorities
trump
trade dispute
beijing
kim jong-un
korea
’ s leader
state media
trade war
u.s.
china
comedic duo
desus
daniel baker
mero
joel martinez
black perspective
malaysia
’ s
new government
harsh laws
criminal charges
small businesses
insurance plans
care
three-year restoration project
mariposa
giant sequoias
yosemite
national park
’ s
trump
phone records
york
reporter shows
federal program
consumer advocates
great gift ideas
wirecutter
stopping
new urgency
philippines
rafael márquez
best-known players
mexico
’ s team
treasury department
drug trafficker
artificial intelligence system ’ s
college debate champion
new gains
jahseh onfroy
xxxtentacion
recent surge
popularity —
album —
violent crimes
charles rennie mackintosh
’ s
glasgow
art
architectural jewel
chad
undo decades
gov
andrew m. cuomo
state le

In [18]:
for story in stories_to_read:
    text_story = TextBlob(story)
    print text_story.noun_phrases

[u'trump', u'bipartisan outcry', u'mr. trump']
[u'jeff sessions', u'stephen miller', u'president \u2019 s', u'top policy adviser', u'\u2019 re', u'immigrant children']
[u'news site', u'propublica', u'central american children', u'immigration authorities']
[u'trump', u'trade dispute', u'beijing']
[u'kim jong-un', u'korea', u'\u2019 s leader', u'state media', u'trade war', u'u.s.', u'china']
[u'comedic duo', u'desus', u'daniel baker', u'mero', u'joel martinez', u'black perspective']
[u'malaysia', u'\u2019 s', u'new government', u'harsh laws', u'criminal charges']
[u'small businesses', u'insurance plans', u'care']
[u'three-year restoration project', u'mariposa', u'giant sequoias', u'yosemite', u'national park']
[u'\u2019 s']
[u'trump', u'phone records', u'york', u'reporter shows']
[u'federal program', u'consumer advocates']
[u'great gift ideas', u'wirecutter']
[u'stopping', u'new urgency', u'philippines']
[u'rafael m\xe1rquez', u'best-known players', u'mexico', u'\u2019 s team', u'treasur

In [43]:
#Filter to find stories on immigration
immig_roots = ['immig', 'sessions', 'border', 'migran']
overall_summary = {}
for story in stories_to_read:
    text_story = TextBlob(story)
    summary = {}
    for phrase in immig_roots:
        if str(text_story.noun_phrases).find(phrase) >0:
            summary['nouns'] = text_story.noun_phrases
            summary['polarity'] = text_story.sentiment.polarity
            summary['subjectivity'] = text_story.sentiment.subjectivity
            overall_summary[story] = summary
pprint.pprint(overall_summary)

{u'\n        Attorney General Jeff Sessions and Stephen Miller, the president\u2019s top policy adviser, once shouted from the sidelines. Now they\u2019re the driving force behind the policy separating immigrant children from parents.    ': {'nouns': WordList([u'jeff sessions', u'stephen miller', u'president \u2019 s', u'top policy adviser', u'\u2019 re', u'immigrant children']),
                                                                                                                                                                                                                                                  'polarity': 0.05000000000000001,
                                                                                                                                                                                                                                                  'subjectivity': 0.5666666666666667},
 u'The audio recording, released by the news site ProPublica, c

In [35]:
#Filter for stories on immigration
immigration_stories = [value for key, value in summary.items() if 'immigration' in value]
print immigration_stories

[]
